# Mapa con datos obtenidos de los espectros $\omega$-k

Luego de descargar los espectros del [ftp](ftp://ecco.jpl.nasa.gov/ECCO2/LLC4320/global/spectral_analysis/) y almacenarnos en mi Google Drive de CICESE, se leyó la información de todos ellos para mostrarla en un mapa interactivo.

Objetivos:
- Saber interpretar la información incluida en el nombre del archivo (e.g. `<season>/<variable>/Spectrum_<id>_at_LON_<lon>_LAT_<lat>.nc`). Es decir, saber la variable para la que se calcula el espectro y a qué temporada corresponde, así como su ubicación en el mapa, la cual suponemos corresponde al centro del 'cuadro' que encierra la región de donde se obtuvo dicho espectro. Este es el principal objetivo, ya que esto nos ayudará a seleccionar visualmente los espectros para las CLO (u otras regiones de interés) que estudiaremos.
- Tener, de ser posible, alguna primera impresión sobre la distribución espacial de las variables
- Poder usar esto como herramienta para seleccionar dinámicamente (con un simple click o menú) los espectros a usar.

In [1]:
from spectral_analysis.common_vars.directories import DATA_FOLDER
prnt = '{}/by_region'.format(DATA_FOLDER)
#tools_fn = 'spectral_analysis_tools.py'
#from shutil import copyfile
#from importlib import reload

In [2]:
# Para leer los datos
import json
import pandas as pd
import geopandas as gpd
# Para el mapa
import branca
import folium
from folium.plugins import MousePosition

In [3]:
# Leemos el archivo que asocia cada espectro (por su ID) con una varible calculada a partir de él
# -- esta variable es inicialmente simple, para propósitos de este demo
season = 'JFM' # ['ASO','JFM']
var = 'KE'    # ['DIV','KE','RV','SSH']
prop = 'sum' # ['avg','rms','stdev','sum']
data = pd.read_csv('{}/{}_{}_{}.csv'.format(prnt,var,season,prop))
data.head()

,Id,sum
0,100,534.455872
1,101,714.876160
2,102,285.611786
3,103,279.546844
4,104,266.671387


In [4]:
# Leemos el archivo que contiene los polígonos de cada región (cuadrados de 6º de lado, pero con alto variable ya que no están equiespaciados en latitud)
geodata = gpd.read_file('{}/{}_{}_geo.json'.format(prnt,var,season),driver='GeoJSON')
geodata.head()

,var,season,lat,lon,s_id,height_km,width_km,geometry
0,KE,JFM,-64.2,-125.0,100,287.888889,290.154066,"POLYGON ((-128 -65.49549999999998, -128 -62.90..."
1,KE,JFM,-64.2,-119.0,101,287.888889,290.154066,"POLYGON ((-122 -65.49549999999998, -122 -62.90..."
2,KE,JFM,-64.2,-113.0,102,287.888889,290.154066,"POLYGON ((-116 -65.49549999999998, -116 -62.90..."
3,KE,JFM,-64.2,-107.0,103,287.888889,290.154066,"POLYGON ((-110 -65.49549999999998, -110 -62.90..."
4,KE,JFM,-64.2,-101.0,104,287.888889,290.154066,"POLYGON ((-104 -65.49549999999998, -104 -62.90..."


In [5]:
geodata = geodata.set_index('s_id',drop=False).join(data.set_index('Id'))
#geodata.sort_values(by=['lat','lon']).head()
min_v, max_v = geodata[prop].quantile([0.05,0.95])#.apply(lambda x: round(x, 2))
#print(min_v,max_v)
#print(geodata[prop].quantile([0.2,0.4,0.6,0.8]))
geodata[prop+'_str']=geodata[prop].apply(lambda x: "{:0.3g}".format(x)) # Formatting - tooltip
geodata.head()

,var,season,lat,lon,s_id,height_km,width_km,geometry,sum,sum_str
s_id,,,,,,,,,,
100,KE,JFM,-64.2,-125.0,100,287.888889,290.154066,"POLYGON ((-128 -65.49549999999998, -128 -62.90...",534.455872,534
101,KE,JFM,-64.2,-119.0,101,287.888889,290.154066,"POLYGON ((-122 -65.49549999999998, -122 -62.90...",714.876160,715
102,KE,JFM,-64.2,-113.0,102,287.888889,290.154066,"POLYGON ((-116 -65.49549999999998, -116 -62.90...",285.611786,286
103,KE,JFM,-64.2,-107.0,103,287.888889,290.154066,"POLYGON ((-110 -65.49549999999998, -110 -62.90...",279.546844,280
104,KE,JFM,-64.2,-101.0,104,287.888889,290.154066,"POLYGON ((-104 -65.49549999999998, -104 -62.90...",266.671387,267


In [6]:
# Colorbar
cmap = branca.colormap.LinearColormap(
    colors=['#fcfbfd','#efedf5','#dadaeb','#bcbddc','#9e9ac8','#807dba','#6a51a3','#54278f','#3f007d'],
    #index=geodata[prop].quantile([0.2,0.4,0.6,0.8]),
    vmin=min_v,
    vmax=max_v,
    caption='{} ({}), season: {}'.format(var,prop.upper(),season)
)

style_fn = lambda x: {
    'fillColor': cmap(x['properties'][prop]),
    'color': 'black',
    'weight':2,
    'fillOpacity':0.2,
    'opacity':0.2
}

cmap

In [7]:
## Mapa

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
m = folium.Map(location=[0,0], zoom_start=3)

data_layer = folium.GeoJson(
    geodata,
    name=var,
    style_function=style_fn,
    tooltip=folium.GeoJsonTooltip(
        fields=['s_id','lat','lon',prop+'_str','height_km','width_km'],
        aliases=['Id','Lat','Lon','{} ({})'.format(var,prop.upper()),'Heigth (km)','Width (km)'],
        localize=True
    )
).add_to(m)

#folium.Choropleth(
#    #geo_data='{}/{}_{}_geo.json'.format(prnt,var,season),
#    geo_data=geo_data,
#    name='choropleth',
#    data=data,
#    columns=['Id','avg'],
#    key_on='feature.id',
#    fill_color='YlGn',
#    fill_opacity=0.2,
#    line_opacity=0.2,
#    legend_name='{} avg'.format(var),
#    control_scale=True
#).add_to(m)


m.add_child(folium.TileLayer('Stamen Terrain'))
m.add_child(folium.TileLayer('OpenStreetMap'))
m.add_child(folium.TileLayer('Stamen Toner'))
m.add_child(folium.LatLngPopup())
m.add_child(MousePosition())
m.add_child(cmap)
m.add_child(folium.LayerControl())

m

## Notas 

### Espectros repetidos

Luego de ver repetidas veces el mapa y revisar los archivos de donde se extrajo la información, puede notarse que hay algunos datos repetidos para la misma zona. 

El siguiente bloque nos muestra la lista de los repetidos.

In [8]:
# Show repeated (lat,lon) pairs
geodata.loc[geodata.duplicated(['lat','lon'],keep=False)].sort_values(by=['lat','lon'])

,var,season,lat,lon,s_id,height_km,width_km,geometry,sum,sum_str
s_id,,,,,,,,,,
201,KE,JFM,-58.568,-59.0,201,345.666667,347.657509,"POLYGON ((-62 -60.12349999999999, -62 -57.0125...",613.828735,614
221,KE,JFM,-58.568,-59.0,221,345.666667,347.657509,"POLYGON ((-62 -60.12349999999999, -62 -57.0125...",3733.083984,3.73e+03
735,KE,JFM,21.612,-59.0,735,568.111111,619.799577,"POLYGON ((-62 19.0555, -62 24.16849999999999, ...",1740.562866,1.74e+03
744,KE,JFM,21.612,-59.0,744,568.111111,619.799577,"POLYGON ((-62 19.0555, -62 24.16849999999999, ...",1067.043091,1.07e+03
778,KE,JFM,31.461,-59.0,778,521.222222,568.663747,"POLYGON ((-62 29.1155, -62 33.80649999999999, ...",753.322754,753
789,KE,JFM,31.461,-59.0,789,521.222222,568.663747,"POLYGON ((-62 29.1155, -62 33.80649999999999, ...",1969.846313,1.97e+03
801,KE,JFM,36.055,-59.0,801,499.666667,538.968259,"POLYGON ((-62 33.80649999999999, -62 38.303500...",1680.034058,1.68e+03
811,KE,JFM,36.055,-59.0,811,499.666667,538.968259,"POLYGON ((-62 33.80649999999999, -62 38.303500...",2532.724854,2.53e+03
823,KE,JFM,40.411,-59.0,823,468.333333,507.609242,"POLYGON ((-62 38.30350000000001, -62 42.5185, ...",1827.792358,1.83e+03


Como puede verse, lo que tienen estos puntos en común es que todos están sobre la longitud -59.

### Espectros que caen en regiones con tierra (islas o continente)

No se sabe si esto se debe a un error de mi parte en la interpretación de a qué punto corresponde (lat,lon) en el nombre del archivo, pero parece haber varios puntos que contienen porciones de tierra. Estos espectros son:


| ID        | Latitud           | Longitud |
| ------------- |:-------------:| -----:|
| 59   | -52.003 | -59 |
| 668 |  5.5552 | -59 |
| 691 |  11.032 | -59 |
| 716 |  16.398 | -59 |
| 844 |  44.521 | -59 |
| 861 |  48.383 | -59 |



---



### Relación entre ambas observaciones

Todos los puntos mencionados anteriormente están sobre la longitud -59, lo cual parece mostrar una relación entre las dos notas.

Si estas dos anomalías son solo para esta longitud, posiblemente no marque ninguna diferencia ya que ahí no hay ninguna CLO. Sin embargo, habría que asegurarse de ser posible.
